In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.2 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
import pandas as pd
import openai
import os
import json


dataset = load_dataset("inkoziev/jokes_dialogues")

df = dataset['train'].to_pandas()

selected_jokes = df.sample(n=50)

def convert_to_chat_format(df):
    chat_format = []
    for _, row in df.iterrows():
        chat_format.append({
            "messages": [
                {"role": "system", "content": "You are a humorous assistant."},
                {"role": "user", "content": row['context']},
                {"role": "assistant", "content": row['utterance']}
            ]
        })
    return chat_format

formatted_data = convert_to_chat_format(selected_jokes)

with open("jokes_for_finetuning_chat.jsonl", "w", encoding="utf-8") as f:
    for item in formatted_data:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

jokes_dialog_dataset.json:   0%|          | 0.00/61.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/114579 [00:00<?, ? examples/s]

In [4]:
!curl https://api.openai.com/v1/files \
  -H "Authorization: Bearer $OPENAI_API_KEY" \
  -F purpose="fine-tune" \
  -F file="@/content/jokes_for_finetuning_chat.jsonl"


{
  "object": "file",
  "id": "file-9TCqHBAxcVcbHskn7ayCA3",
  "purpose": "fine-tune",
  "filename": "jokes_for_finetuning_chat.jsonl",
  "bytes": 28049,
  "created_at": 1741800208,
  "expires_at": null,
  "status": "processed",
  "status_details": null
}


In [5]:
from openai import OpenAI

client = OpenAI()

client.fine_tuning.jobs.create(
    training_file="file-9TCqHBAxcVcbHskn7ayCA3",
    model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-7US8lSf2q4Bskh654xWnDUgy', created_at=1741800225, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=[], seed=1212996906, status='validating_files', trained_tokens=None, training_file='file-9TCqHBAxcVcbHskn7ayCA3', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None, metadata=None)

In [11]:
fine_tuning_job = client.fine_tuning.jobs.retrieve("ftjob-7US8lSf2q4Bskh654xWnDUgy")
print(fine_tuning_job)

FineTuningJob(id='ftjob-7US8lSf2q4Bskh654xWnDUgy', created_at=1741800225, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-4o-mini-2024-07-18:personal::BAKF0Cpu', finished_at=1741800785, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-yyyPGCw9VCJjgaycEyhn8xFf', result_files=['file-XGTzgFVGCmR8TawnBhLh3h'], seed=1212996906, status='succeeded', trained_tokens=14112, training_file='file-9TCqHBAxcVcbHskn7ayCA3', validation_file=None, estimated_finish=None, integrations=[], method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=1.8, n_epochs=3)), type='supervised'), user_provided_suffix=None, metadata=None)


In [12]:
client = openai.OpenAI()

completion = client.chat.completions.create(
    model="ft:gpt-4o-mini-2024-07-18:personal::BAKF0Cpu",
    messages=[
        {"role": "system", "content": "Ты анекдотный мастер, продолжи фразу остроумным анекдотом"},
        {"role": "user", "content": "Раз яблоко, два яблоко"}
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Спугнули барашка.', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None, annotations=[])
